In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.events"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Getting the upcoming 10 events
No upcoming events found.


In [2]:
def create_event(service, summary, location, description, start_time, end_time):
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': 'America/New_York',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'America/New_York',
        },
    }

    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f'Event created: {event.get("htmlLink")}')


In [3]:
def update_event(service, date, summary, new_location=None, new_description=None):
    # Get all events
    events_result = service.events().list(calendarId='primary', singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])

    # Find the event that starts on the specified date and has the specified summary
    for event in events:
        start_time = event['start'].get('dateTime', event['start'].get('date'))
        event_summary = event.get('summary', '')
        if start_time.startswith(date) and event_summary == summary:
            # Get the event ID
            event_id = event['id']

            # Update the event
            event = service.events().get(calendarId='primary', eventId=event_id).execute()

            if new_location is not None:
                event['location'] = new_location
            if new_description is not None:
                event['description'] = new_description

            updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event).execute()

            print(f'Event updated: {updated_event.get("htmlLink")}')
            return

    print('No event found on the specified date and summary')

In [4]:
def main():
    # ... (Your existing code)

    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("calendar", "v3", credentials=creds)

        # ... (Your existing code)

    except HttpError as error:
        print(f"An error occurred: {error}")
        return

    # Call the create_event function
    create_event(service, 'Meeting', 'ccc', 'discussion1','2024-03-15T10:00:00-05:00', '2024-03-15T11:00:00-05:00')
    update_event(service, '2024-03-15', 'Meeting', 'bbb', 'discussion2')


if __name__ == "__main__":
    main()

Event created: https://www.google.com/calendar/event?eid=MGs1ZnNiamNobXU1djNnaWs5ZDY1NnNwdDQgeGluaHd1QHVtaWNoLmVkdQ
Event updated: https://www.google.com/calendar/event?eid=MGs1ZnNiamNobXU1djNnaWs5ZDY1NnNwdDQgeGluaHd1QHVtaWNoLmVkdQ
